# SQL Data Manipulation

## Case Statements

`WHEN`, `THEN`, `ELSE`, `END`

#### CASE in SELECT 

```postgresql
SELECT
    CASE WHEN criteria THEN display_message
         WHEN criteria THEN display_message
         ELSE display_message
         END AS field_name
FROM db1;
```

#### CASE in WHERE

```postgresql
SELECT field
FROM db1
WHERE CASE WHEN criteria THEN display_message
      CASE WHEN criteria THEN display_message
      END IS NOT NULL;
```


#### CASE with aggregate functions

```postgresql
SELECT
    SUM(CASE WHEN criteria THEN value_field END) AS s,
    COUNT(CASE WHEN criteria THEN display_message END) AS c
    ROUND(AVG(CASE WHEN criteria THEN 1 
                   WHEN criteria THEN 0),2) as percentage_avg
FROM db1
GROUP BY field

```

## Subquery

Returns intermediary transformed information of a scalar, a list, or a table

#### Subquery in WHERE
```postgresql
SELECT field
FROM db1
WHERE field_1 IN
    (SELECT field_1
     FROM db2
     WHERE field_2 = 5);

```


#### Subquery in FROM
```postgresql
SELECT field
FROM (SELECT field
      FROM db1
      WHERE criteria)

```


#### Subquery in SELECT
**Has to be a single value**

```postgresql
SELECT field, 
        (SELECT field
         FROM db2
         WHERE criteria)
FROM db1;

```


## Common Table Expressions (CTEs)

Declare a table before main query and use it in `FROM` statement

1. CTEs are stored in the memory after execution, improving performance
2. CTEs provide better readability than subquery

```postgresql
-- CTEs:
WITH tb1 AS (
    SELECT field
    FROM db1
    WHERE criteria),
tb2 AS (
    SELECT field
    FROM db1
    WHERE criteria)
-- Main query starts below:
SELECT
FROM
WHERE

```

## Window Functions


### Basics

#### OVER()
Used in `SELECTION` to simplify subquery by implying `FROM`,`WHERE`... statements of main query

Instead of writing:
```postgresql
SELECT field_1, 
       (SELECT AVG(field_1)
        FROM tb1
        WHERE criteria) AS average
FROM tb1
WHERE criteria
```

Can now write:
```postgresql
SELECT field_1, AVG(field_1) OVER() AS average
FROM tb1
WHERE criteria
```



#### OVER(ORDER BY)

```postgresql
SELECT ROW_NUMBER() OVER(ORDER BY filed DESC) AS row_n
```



#### OVER(PARTITION BY)

```postgresql
SELECT AVG(field_1 + field_2) OVER(PARTITION BY field) AS field_avg
```


### Ranking

#### ROW_NUMBER()

Assigns number to rows regardless if row values are unique or not

```postgresql
SELECT ROW_NUMBER() OVER() as row_n
FROM tb1

```



#### RANK()

Assigns same number to rows sharing identical value, and skip ranks by the number of identical value

value: 1,2,2,3

ranking: 1,2,2,4

```postgresql
SELECT RANK() OVER(ORDER BY AVG(field_1 + field_2)DESC) AS field_rank
```


#### DENSE_RANK()

Assigns same number to rows sharing identical value without skipping

value: 1,2,2,3

ranking: 1,2,2,3

### Fetching


#### LAG()

`LAG(field, n)` returns `field` value at the row `n` before the current row
```postgresql
LAG(field, num_row_downshift) OVER(ORDER BY field) AS _
```



#### LEAD()

`LAG(field, n)` returns `field` value at the row `n` after the current row
```postgresql
LAG(field, num_row_upshift) OVER(ORDER BY field) AS _
```



#### FIRST_VALUE()

`FIRST_VALUE(field)` returns the first value of `field` in the table or partition



#### LAST_VALUE()

`LAST_VALUE(field)` returns the last value of `field` in the table or partition


### Paging

Split data into (approximately) equal chunks

#### NTILE()

`NTILE(n)` split table into `n` (approximately) equal pages

```postgresql
SELECT NTILE(5) OVER() AS page

```

### Framing

#### ROWS BETWEEN

`ROWS BETWEEN [start] AND [finish]`

`[start]`, `[finish]` can be:

- PRECEDING
- FOLLOWING
- UNBOUNDED PRECEDING
- UNBOUNDED FOLLOWING
- CURRENT ROW

```postgresql
SELECT SUM(field) OVER(ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total;

SELECT AVG(field) OVER(ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS three_moving_average;

```

#### RANGE BETWEEN

Treats duplicates as a single entity. Less often compared to `ROWS BETWEEN`

### Pivoting


```postgresql
CREATE EXTENSION IF NOT EXISTS tablefunc

SELECT * 
FROM CROSSTAB($$
    SELECT
        field_1, field_2, COUNT(*) :: INTEGER AS filed_3
    FROM tb1
    WHERE criteria
$$) AS ct(field_1 VARCHAR, field_2 VARCHAR, field_3 INTEGER)
```

### ROLLUP & CUBE

#### ROLLUP()

Subclause of GROUP BY to add extra row that represents group-level aggregation

```postgresql
SELECT COUNT(*)
FROM db1
GROUP BY field_1 ROLLUP(field_2)
```

Will add extra row that calculate the total count grouped by `field_1`

#### CUBE()

Add extra rows to ROLLUP()

### Fill Nulls

```postgresql
SELECT COALESCE(field, fill_value) AS field_1

```

### Compress Data

Join all values in a field into a long string with `delimiter`

```postgresql
STRING_AGG(field, delimiter)
```